In [1]:
! pip install optuna seqeval evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incom

In [2]:
import pandas as pd
import ast
from tqdm import tqdm
from collections import Counter
import datasets
from datasets import (Dataset, Features, Sequence, Value, ClassLabel, load_dataset,
                    load_from_disk, concatenate_datasets, DatasetDict)
from sklearn.model_selection import KFold
from transformers import (AutoTokenizer, AutoModel, AutoModelForTokenClassification,
                         pipeline, TrainingArguments, Trainer,
                         DataCollatorForTokenClassification, EarlyStoppingCallback)
import torch
import optuna
from optuna.pruners import MedianPruner
import os
os.environ['WANDB_DISABLED'] = 'true'
import pickle
import numpy as np
import seqeval
import evaluate
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

seqeval = evaluate.load("seqeval")

2025-09-26 17:49:50.481839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758908990.854228      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758908990.957147      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [4]:
TRAIN_PATH_COLAB = "/content/drive/MyDrive/Hackatons/X5_ner_2025/train.csv"
SUBMISSION_PATH_COLAB = "/content/drive/MyDrive/Hackatons/X5_ner_2025/submission.csv"
TRAIN_PATH_KAGGLE = "/kaggle/input/x5-ner-train/train.csv"
TRAIN_AUGMENTED = "/kaggle/input/train-augmented-col-perc/train_with_augmented_volume_percent.csv"

## IN

## Подготовка датасета

In [5]:
train_df_raw = pd.read_csv(TRAIN_AUGMENTED, sep=';')
train_df_raw

,sample,annotation
0,aa,"[(0, 2, 'O')]"
1,aala,"[(0, 4, 'O')]"
2,aarcca,"[(0, 6, 'O')]"
3,abon,"[(0, 4, 'O')]"
4,abso,"[(0, 4, 'B-BRAND')]"
...,...,...
27547,ветчина 300 гр нарезка,"[(0, 7, 'B-TYPE'), (8, 11, 'B-VOLUME'), (12, 1..."
27548,кукуруза 400 г mikado,"[(0, 8, 'B-TYPE'), (9, 12, 'B-VOLUME'), (13, 1..."
27549,кукуруза 340 гр,"[(0, 8, 'B-TYPE'), (9, 12, 'B-VOLUME'), (13, 1..."
27550,хлеб 350 г 5 злаков,"[(0, 4, 'B-TYPE'), (5, 8, 'B-VOLUME'), (9, 10,..."


In [6]:
train_df_raw['annotation'] = train_df_raw['annotation'].str.replace("\'0\'", "O")

In [7]:
train_df_raw['annotation'] = train_df_raw['annotation'].apply(lambda x: ast.literal_eval(str(x)))

In [8]:
all_words = []
all_tags = []
for i, row in tqdm(train_df_raw.iterrows()):
    words_sample = []
    entities_sample = []
    for ent in row['annotation']:
        word = row['sample'][ent[0]:ent[1]]
        words_sample.append(word)
        entities_sample.append(ent[2])
    all_words.append(words_sample)
    all_tags.append(entities_sample)
    # print(words_sample, entities_sample)

27552it [00:01, 26380.00it/s]


In [9]:
assert len(all_words) == len(all_tags), 'different lengths'

In [10]:
lbls_in_dataset = [
 'O',
 'B-BRAND',
 'B-PERCENT',
 'B-TYPE',
 'B-VOLUME',
 'I-BRAND',
 'I-PERCENT',
 'I-TYPE',
 'I-VOLUME']
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

In [11]:
features=Features(
    {
        "id": Value(dtype='int32', id=None),
        "tokens": Sequence(feature=Value(dtype='string', id=None)),
        "ner_tags": Sequence(feature=ClassLabel(num_classes=len(lbls_in_dataset), names=list(lbls_in_dataset)), id=None)
    }
)

In [12]:
ds = Dataset.from_dict({"id": list(range(len(all_words))),
                        "tokens": all_words,
                        "ner_tags": all_tags},
                       features=features)

In [ ]:
ds_splitted = ds.train_test_split(
    test_size=0.25, shuffle=True, seed=42,
    # stratify_by_column='ner_tags'
)

In [43]:
Counter([x for l in ds['ner_tags'] for x in l])

Counter({0: 5407,
         1: 7252,
         3: 24845,
         5: 490,
         7: 4703,
         2: 180,
         4: 207,
         8: 163,
         6: 15})

In [ ]:
Counter([x for l in ds_splitted['test']['ner_tags'] for x in l])

Counter({3: 6129, 1: 1774, 7: 1133, 0: 1358, 5: 122, 2: 9, 4: 18, 8: 8, 6: 2})

## Загрузка модели

In [13]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2',
                                          use_fast=True,
                                          add_prefix_space=True)

# model = AutoModelForTokenClassification.from_pretrained('/kaggle/input/rut5_large_ft_colab_2509/pytorch/default/1/ruT5_large_250925',
#                                                         num_labels=len(lbls_in_dataset),
#                                                         id2label=id2label,
#                                                         label2id=label2id).to("cuda")

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
s = '''
сироп топинамбура
'''
r = tokenizer(s)
[tokenizer.decode(x) for x in r.input_ids][:5]

['сироп', 'то', 'пина', 'м', 'бур']

## Токенизация и подготовка

In [14]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_index = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
tokenized_ds_train = ds_splitted['train'].map(tokenize_and_align_labels,
                                              batched=True,
                                              fn_kwargs={'tokenizer': tokenizer})

Map:   0%|          | 0/20438 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
tokenized_ds_test = ds_splitted['test'].map(tokenize_and_align_labels,
                                              batched=True,
                                              fn_kwargs={'tokenizer': tokenizer})

Map:   0%|          | 0/6813 [00:00<?, ? examples/s]

## Обучение

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

print("Compiling model for faster training...")
torch.set_float32_matmul_precision('high')
model.to(device)

Using device: cuda:0
Compiling model for faster training...


T5ForTokenClassification(
  (transformer): T5EncoderModel(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo):

In [ ]:
# функция метрики
def compute_metrics_custom(p):
    labels_list = list(id2label.values())
    predictions, labels = p
    if not os.path.exists('/content/test/p_trainer.pkl'):
        os.makedirs('/content/test', exist_ok=True)
        with open('/content/test/p_trainer.pkl', 'wb') as f:
            pickle.dump(p, f)
    predictions = np.argmax(predictions, axis=2)


    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    report_dict = classification_report(true_labels, true_predictions, digits=4, output_dict=True)
    report = classification_report(true_labels, true_predictions, digits=4)
    macro_f1 = report_dict["macro avg"]["f1-score"]
    print("=== seqeval classification_report ===")
    print(report)
    CLASS_REPORT_PATH = '/content/logs/last_classification_report.txt'
    try:
        with open(CLASS_REPORT_PATH, "w", encoding="utf-8") as f:
            f.write(report)
    except Exception as e:
        print(f"Warning: failed to write classification report to {CLASS_REPORT_PATH}: {e}")

    return {
        "f1_macro": macro_f1,
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import optuna.logging
optuna.logging.set_verbosity(optuna.logging.INFO)

best_f1 = 0

# GDRIVE_DIR = '/content/drive/MyDrive/Hackatons/X5_ner_2025/ruT5_large_250925'

def printer(s):
    print('*'*150, end='\n\n')
    print(s, end='\n\n')
    print('*'*150, end='\n\n')

In [ ]:
early_stop_cb = EarlyStoppingCallback(early_stopping_patience=2)

In [ ]:
def objective(trial: optuna.Trial, model=model):
    global best_f1
    model = model
    model_name = 'ruT5_large_250925_optuna_v1'
    trial_check_dir = f"./checkpoints_trial"

    os.makedirs(trial_check_dir, exist_ok=True)
    args = TrainingArguments(
        # f"{model_name}-finetuned-ner",
        output_dir=trial_check_dir,
        overwrite_output_dir=True,
        eval_strategy = "epoch",
        torch_compile=True,
        # 64\
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
        weight_decay=trial.suggest_loguniform('weight_decay', 1e-4, 0.05),
        num_train_epochs=trial.suggest_int('num_train_epochs', low = 3, high = 10),
        seed=42,
        data_seed=24,
        gradient_accumulation_steps=2,
        warmup_ratio=0.1,
        report_to=None,
        logging_dir="./logs",
        logging_steps=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1_macro",
        # greater_is_better=False,
        save_total_limit=1,
        save_strategy="epoch",  # Changed to match evaluation_strategy
    )

    # early_stopping = EarlyStoppingCallback(
    #     early_stopping_patience=1,  # Stop if F1 decreases for 1 consecutive epoch
    #     early_stopping_threshold=0.001
    # )

    trainer = Trainer(
        model,
        args,
        train_dataset=tokenized_ds_train,
        eval_dataset=tokenized_ds_test,
        # train_dataset=small_dataset_train,
        # eval_dataset=small_dataset_test,
        data_collator=data_collator,
        compute_metrics=compute_metrics_custom,
        tokenizer=tokenizer,
        callbacks=[early_stop_cb],
    )

    trainer.train()

    # Evaluate and save best model globally
    eval_metrics = trainer.evaluate()
    current_f1 = eval_metrics["eval_f1_macro"]

    if current_f1 > best_f1:
        best_f1 = current_f1
        trainer.save_model("./best_model")
        trainer.save_model(GDRIVE_DIR)
        printer(f"New best model saved with F1: {best_f1:.4f}")
    return current_f1

In [ ]:
study = optuna.create_study(study_name='test_optuna', direction='maximize')
study.optimize(func=objective, n_trials=5)

[I 2025-09-25 15:47:56,064] A new study created in memory with name: test_optuna
/tmp/ipython-input-992928850.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
/tmp/ipython-input-992928850.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform('weight_decay', 1e-4, 0.05),
The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_

Epoch,Training Loss,Validation Loss,F1 Macro,Precision,Recall,F1,Accuracy
1,0.742300,0.537095,0.400311,0.803884,0.892531,0.845891,0.843418
2,0.362400,0.358328,0.450653,0.905844,0.943039,0.924067,0.915001
3,0.158600,0.302577,0.579675,0.930021,0.954029,0.941872,0.930194
4,0.158400,0.325737,0.720133,0.916179,0.951784,0.933642,0.924071
5,0.033900,0.308182,0.823673,0.938384,0.958659,0.948413,0.937526
6,0.041600,0.379980,0.770415,0.939387,0.954543,0.946905,0.936732
7,0.036000,0.425532,0.874428,0.938848,0.960670,0.949634,0.938206
8,0.014800,0.478993,0.876659,0.943126,0.959313,0.951151,0.938773
9,0.004200,0.514241,0.876177,0.942816,0.958425,0.950557,0.938093


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.6379    0.7954    0.7080      5043
     PERCENT     0.0000    0.0000    0.0000        14
        TYPE     0.8637    0.9250    0.8933     16297
      VOLUME     0.0000    0.0000    0.0000        29

   micro avg     0.8039    0.8925    0.8459     21383
   macro avg     0.3754    0.4301    0.4003     21383
weighted avg     0.8087    0.8925    0.8478     21383



/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch,Training Loss,Validation Loss,F1 Macro,Precision,Recall,F1,Accuracy
1,0.009600,0.497267,0.858961,0.939611,0.957583,0.948512,0.936846
2,0.001700,0.623256,0.883508,0.939863,0.956741,0.948227,0.936392
3,0.000200,0.633704,0.879540,0.940540,0.958706,0.949536,0.937866


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9097    0.9014    0.9056      5043
     PERCENT     1.0000    0.6429    0.7826        14
        TYPE     0.9487    0.9756    0.9619     16297
      VOLUME     0.8148    0.7586    0.7857        29

   micro avg     0.9396    0.9576    0.9485     21383
   macro avg     0.9183    0.8196    0.8590     21383
weighted avg     0.9394    0.9576    0.9483     21383

=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9054    0.9032    0.9043      5043
     PERCENT     1.0000    0.6429    0.7826        14
        TYPE     0.9504    0.9736    0.9619     16297
      VOLUME     0.8438    0.9310    0.8852        29

   micro avg     0.9399    0.9567    0.9482     21383
   macro avg     0.9249    0.8627    0.8835     21383
weighted avg     0.9397    0.9567    0.9481     21383

=== seqeval classification_report ===
              pr

There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight'].


[I 2025-09-25 17:14:40,644] Trial 4 finished with value: 0.8835080653340189 and parameters: {'learning_rate': 0.00012973534241957156, 'weight_decay': 0.0006306410098490446, 'num_train_epochs': 3}. Best is trial 2 with value: 0.8926324166140165.


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9054    0.9032    0.9043      5043
     PERCENT     1.0000    0.6429    0.7826        14
        TYPE     0.9504    0.9736    0.9619     16297
      VOLUME     0.8438    0.9310    0.8852        29

   micro avg     0.9399    0.9567    0.9482     21383
   macro avg     0.9249    0.8627    0.8835     21383
weighted avg     0.9397    0.9567    0.9481     21383



In [ ]:
best_trial = study.best_trial
printer(f"Final best F1: {best_trial.value}")
printer(f"Final params: {best_trial.params}")

******************************************************************************************************************************************************

Final best F1: 0.8926324166140165

******************************************************************************************************************************************************

******************************************************************************************************************************************************

Final params: {'learning_rate': 6.44944805049113e-05, 'weight_decay': 0.004066034568680406, 'num_train_epochs': 9}

******************************************************************************************************************************************************



## тест

In [ ]:
df_test = pd.read_csv('submission.csv', sep=';', usecols=['sample'])

In [ ]:
best_model_path = '/content/drive/MyDrive/Hackatons/X5_ner_2025/ruT5_large_250925'
final_tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=True, add_prefix_space=True)
final_model = AutoModelForTokenClassification.from_pretrained(best_model_path)


In [ ]:
final_model.to('cuda')

T5ForTokenClassification(
  (transformer): T5EncoderModel(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo):

In [ ]:
token_classifier = pipeline(
    "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
)
for s in samples:
    print(f'sample: {s}')
    res = token_classifier(s)
    for i, r in enumerate(res):
        # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
        print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '. Probs:  ' + str(round(r['score'], 4)))
    print('#'*40)

Device set to use cuda:0


NameError: name 'samples' is not defined

In [ ]:
s = 'очиститель для унитаза'
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '. Probs:  ' + str(round(r['score'], 4)))
print(res)

Entity: TYPE   Word: очиститель. Probs:  0.9998
[{'entity_group': 'TYPE', 'score': np.float32(0.9997584), 'word': 'очиститель', 'start': 0, 'end': 10}]


In [ ]:
lbls_in_dataset = [
 'O',
 'B-BRAND',
 'B-PERCENT',
 'B-TYPE',
 'B-VOLUME',
 'I-BRAND',
 'I-PERCENT',
 'I-TYPE',
 'I-VOLUME']
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

In [21]:
import torch
import torch.nn.functional as F
import re

def predict_all_entities(text: str, model, tokenizer, id2label, device=None, debug=False):
    """
    Word-level inference with original character spans (start_idx, end_idx, ENTITY).
    Returns a list of tuples for each word (including 'O').
    """

    model.eval()
    if device is None:
        device = next(model.parameters()).device

    # --- find words and their char spans in original text ---
    words = []
    spans = []
    for match in re.finditer(r"\S+", text):
        words.append(match.group())
        spans.append(match.span())  # (start_idx, end_idx)

    # encode with word-level info
    enc = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)
    word_ids = enc.word_ids(batch_index=0)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[0]               # (seq_len, num_labels)
        probs = F.softmax(logits, dim=-1)        # (seq_len, num_labels)
    # print()
    # print(probs)
    results = []
    prev_word_idx = None
    # print(word_ids)
    for token_idx, word_idx in enumerate(word_ids):
        if debug:
            print(token_idx, word_idx, logits[token_idx])
        if word_idx is None:
            prev_word_idx = None
            continue

        # only take first subtoken per word
        if word_idx != prev_word_idx:
            label_id = int(torch.argmax(logits[token_idx]).cpu().numpy())
            label = id2label[label_id]

            start_idx, end_idx = spans[word_idx]
            results.append((start_idx, end_idx, label))

        prev_word_idx = word_idx

    return results


# -------------------------
# Example usage
# -------------------------
s = "сыр натура сливочный"

# Example id2label (replace with your mapping)
# id2label = {0: "O", 1: "B-TYPE", 2: "I-TYPE", ...}

res = predict_all_entities(s, trained_model, tokenizer, id2label)
print(res)


[(0, 3, 'B-TYPE'), (4, 10, 'B-BRAND'), (11, 20, 'I-BRAND')]


In [ ]:
annotations = []
for s in tqdm(df_test['sample'].tolist()):
    r = predict_all_entities(s, final_model, final_tokenizer, id2label)
    annotations.append(r)

100%|██████████| 5000/5000 [01:59<00:00, 41.73it/s]


In [ ]:
df_test['annotation'] = annotations

In [ ]:
df_test.sample(20)

,sample,annotation
1501,куркум,"[(0, 6, B-TYPE)]"
2586,вялены,"[(0, 6, B-TYPE)]"
2653,exponetto,"[(0, 9, B-BRAND)]"
1055,моцарелла шарики,"[(0, 9, B-TYPE), (10, 16, I-TYPE)]"
705,галеты petra,"[(0, 6, B-TYPE), (7, 12, B-BRAND)]"
106,ванилик,"[(0, 7, B-TYPE)]"
589,нектар j,"[(0, 6, B-TYPE), (7, 8, B-BRAND)]"
2468,тнк,"[(0, 3, O)]"
2413,рыбный фарш,"[(0, 6, B-TYPE), (7, 11, I-TYPE)]"
1600,сгущенные молочн,"[(0, 9, B-TYPE), (10, 16, I-TYPE)]"


In [ ]:
df_test.to_csv('submission_ruT5_large_250925.csv', index=False, sep=';')

## k-fold

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
ds_tokenized = ds.map(tokenize_and_align_labels,
                                          batched=True,
                                          fn_kwargs={'tokenizer': tokenizer})

In [16]:
# функция метрики
def compute_metrics_custom(p):
    labels_list = list(id2label.values())
    predictions, labels = p
    if not os.path.exists('/content/test/p_trainer.pkl'):
        os.makedirs('/content/test', exist_ok=True)
        with open('/content/test/p_trainer.pkl', 'wb') as f:
            pickle.dump(p, f)
    predictions = np.argmax(predictions, axis=2)


    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    report_dict = classification_report(true_labels, true_predictions, digits=4, output_dict=True)
    report = classification_report(true_labels, true_predictions, digits=4)
    macro_f1 = report_dict["macro avg"]["f1-score"]
    print("=== seqeval classification_report ===")
    print(report)
    CLASS_REPORT_PATH = '/content/logs/last_classification_report.txt'
    try:
        with open(CLASS_REPORT_PATH, "w", encoding="utf-8") as f:
            f.write(report)
    except Exception as e:
        print(f"Warning: failed to write classification report to {CLASS_REPORT_PATH}: {e}")

    return {
        "f1_macro": macro_f1,
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
early_stop_cb = EarlyStoppingCallback(early_stopping_patience=2)

In [19]:
# Assuming these are defined globally:
# - full_dataset: your full Hugging Face Dataset (not pre-split)
# - tokenizer, data_collator, compute_metrics_custom
# - num_labels, id2label, label2id
# - early_stop_cb (if used)
# - GDRIVE_DIR, printer

def objective(trial: optuna.Trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-4, 0.05, log=True)
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 10)
    
    # 5-Fold CV setup
    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    fold_f1_scores = []



    # Get indices for all folds
    all_splits = list(kfold.split(ds_tokenized))

    for fold, (train_idx, val_idx) in enumerate(all_splits):
        # Create train/val datasets for this fold
        train_ds = ds_tokenized.select(train_idx)
        val_ds = ds_tokenized.select(val_idx)

        # ✅ CRITICAL: Load FRESH model for each fold
        model = AutoModelForTokenClassification.from_pretrained(
            'ai-forever/ruT5-large',  # or better: switch to ruBERT!
            num_labels=len(lbls_in_dataset),
            id2label=id2label,
            label2id=label2id
        ).to("cuda")

        trial_check_dir = f"./checkpoints_trial_{trial.number}_fold_{fold}"
        os.makedirs(trial_check_dir, exist_ok=True)

        args = TrainingArguments(
            output_dir=trial_check_dir,
            overwrite_output_dir=True,
            eval_strategy="epoch",
            torch_compile=True,
            per_device_train_batch_size=256,        # Reduced from 256 (T5-large is huge!)
            per_device_eval_batch_size=256,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            num_train_epochs=num_train_epochs,
            seed=42 + fold,                        # Different seed per fold
            data_seed=24 + fold,
            gradient_accumulation_steps=1,         # To compensate for smaller batch
            warmup_ratio=0.1,
            report_to=None,
            logging_dir="./logs",
            logging_steps=1,
            load_best_model_at_end=True,
            metric_for_best_model="eval_f1_macro",
            save_total_limit=1,
            save_strategy="epoch",
            # Disable logging to speed up CV
            disable_tqdm=True,
        )

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=val_ds,
            data_collator=data_collator,
            compute_metrics=compute_metrics_custom,
            tokenizer=tokenizer,
            callbacks=[early_stop_cb] if 'early_stop_cb' in globals() else [],
        )

        trainer.train()

        # Evaluate on validation fold
        eval_metrics = trainer.evaluate()
        fold_f1 = eval_metrics["eval_f1_macro"]
        fold_f1_scores.append(fold_f1)

        # Report intermediate value to Optuna for pruning
        trial.report(fold_f1, step=fold)

        # Pruning: check if trial should be pruned
        if trial.should_prune():
            # Cleanup before pruning
            del model, trainer
            torch.cuda.empty_cache()
            raise optuna.TrialPruned()

        # # Cleanup to save memory
        # del model, trainer
        # torch.cuda.empty_cache()

    # Return average F1 across all 5 folds
    avg_f1 = np.mean(fold_f1_scores)
    return avg_f1


# Create study with MedianPruner
study = optuna.create_study(
    study_name='test_optuna_3fold',
    direction='maximize',
    pruner=MedianPruner(
        n_startup_trials=2,    # First 3 trials not pruned
        n_warmup_steps=1,      # Wait for 2 folds before pruning
        interval_steps=1       # Check every fold
    )
)

# Run optimization
study.optimize(objective, n_trials=10)  # Reduced trials due to 5x cost

# After optimization, print best results
print("Best trial:")
print(f"  Value: {study.best_value:.4f}")
print("  Params:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

[I 2025-09-26 17:53:41,250] A new study created in memory with name: test_optuna_5fold


Map:   0%|          | 0/27552 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_36/3364797558.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  war

{'loss': 2.265, 'grad_norm': 8.001017570495605, 'learning_rate': 0.0, 'epoch': 0.022727272727272728}
{'loss': 2.2583, 'grad_norm': 8.146414756774902, 'learning_rate': 8.570529762207097e-06, 'epoch': 0.045454545454545456}
{'loss': 2.2364, 'grad_norm': 8.015944480895996, 'learning_rate': 1.7141059524414193e-05, 'epoch': 0.06818181818181818}
{'loss': 2.203, 'grad_norm': 8.202024459838867, 'learning_rate': 2.571158928662129e-05, 'epoch': 0.09090909090909091}
{'loss': 2.1436, 'grad_norm': 7.650907039642334, 'learning_rate': 3.428211904882839e-05, 'epoch': 0.11363636363636363}
{'loss': 2.0474, 'grad_norm': 7.481686115264893, 'learning_rate': 4.285264881103548e-05, 'epoch': 0.13636363636363635}
{'loss': 1.972, 'grad_norm': 6.931197166442871, 'learning_rate': 5.142317857324258e-05, 'epoch': 0.1590909090909091}
{'loss': 1.845, 'grad_norm': 6.843026638031006, 'learning_rate': 5.9993708335449675e-05, 'epoch': 0.18181818181818182}
{'loss': 1.7292, 'grad_norm': 6.0464653968811035, 'learning_rate': 

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

{'loss': 1.0481, 'grad_norm': 1.3634649515151978, 'learning_rate': 3.149257811294489e-06, 'epoch': 3.022727272727273}
{'loss': 1.0061, 'grad_norm': 1.507120966911316, 'learning_rate': 3.077683770128705e-06, 'epoch': 3.0454545454545454}
{'loss': 1.0298, 'grad_norm': 1.384630560874939, 'learning_rate': 3.006109728962921e-06, 'epoch': 3.0681818181818183}
{'loss': 1.0253, 'grad_norm': 1.3590863943099976, 'learning_rate': 2.934535687797137e-06, 'epoch': 3.090909090909091}
{'loss': 1.044, 'grad_norm': 1.271601676940918, 'learning_rate': 2.8629616466313537e-06, 'epoch': 3.1136363636363638}
{'loss': 1.0168, 'grad_norm': 1.3768301010131836, 'learning_rate': 2.79138760546557e-06, 'epoch': 3.1363636363636362}
{'loss': 1.045, 'grad_norm': 1.270020604133606, 'learning_rate': 2.719813564299786e-06, 'epoch': 3.159090909090909}
{'loss': 0.9907, 'grad_norm': 1.3973199129104614, 'learning_rate': 2.648239523134002e-06, 'epoch': 3.1818181818181817}
{'loss': 1.0546, 'grad_norm': 1.229644536972046, 'learnin

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.8584    0.2344    0.3682      3311
     PERCENT     0.0000    0.0000    0.0000        86
        TYPE     0.6017    0.9173    0.7267     11299
      VOLUME     0.0000    0.0000    0.0000        42

   micro avg     0.6145    0.7559    0.6779     14738
   macro avg     0.3650    0.2879    0.2737     14738
weighted avg     0.6541    0.7559    0.6399     14738

{'eval_loss': 0.9619789123535156, 'eval_f1_macro': 0.2737285599643172, 'eval_precision': 0.6144724504991451, 'eval_recall': 0.7559370335187949, 'eval_f1': 0.677903191456996, 'eval_accuracy': 0.6654281186820392, 'eval_runtime': 1.5205, 'eval_samples_per_second': 3623.734, 'eval_steps_per_second': 7.234, 'epoch': 4.0}
{'train_runtime': 24.85, 'train_samples_per_second': 3548.003, 'train_steps_per_second': 7.082, 'train_loss': 1.3460459069094874, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2025-09-26 18:24:46,353] Trial 9 pruned. 


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.8584    0.2344    0.3682      3311
     PERCENT     0.0000    0.0000    0.0000        86
        TYPE     0.6017    0.9173    0.7267     11299
      VOLUME     0.0000    0.0000    0.0000        42

   micro avg     0.6145    0.7559    0.6779     14738
   macro avg     0.3650    0.2879    0.2737     14738
weighted avg     0.6541    0.7559    0.6399     14738

{'eval_loss': 0.9619789123535156, 'eval_f1_macro': 0.2737285599643172, 'eval_precision': 0.6144724504991451, 'eval_recall': 0.7559370335187949, 'eval_f1': 0.677903191456996, 'eval_accuracy': 0.6654281186820392, 'eval_runtime': 1.5428, 'eval_samples_per_second': 3571.521, 'eval_steps_per_second': 7.13, 'epoch': 4.0}
Best trial:
  Value: 0.9175
  Params:
    learning_rate: 0.00037710330953711225
    weight_decay: 0.040411977783360725
    num_train_epochs: 10


In [24]:
doc[0]

'abon'

In [18]:
df_all_valid_docs

,id,tokens,ner_tags,input_ids,attention_mask,labels,predicted_ner_tags
0,3,[abon],[0],"[8, 4877, 1146, 2]","[1, 1, 1, 1]","[0, 0, 0, -100]","[(0, 3, B-TYPE), (4, 10, I-TYPE), (11, 20, I-T..."
1,6,"[abtoys, игрушк]","[1, 3]","[8, 4877, 2769, 448, 125, 5112, 319, 189, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 3, 3, 3, -100]","[(0, 3, B-TYPE), (4, 10, I-TYPE), (11, 20, I-T..."
2,17,[active],[1],"[8, 4128, 2496, 2]","[1, 1, 1, 1]","[1, 1, 1, -100]","[(0, 3, B-TYPE), (4, 10, I-TYPE), (11, 20, I-T..."
3,29,[agata],[1],"[8, 6084, 12625, 2]","[1, 1, 1, 1]","[1, 1, 1, -100]","[(0, 3, B-TYPE), (4, 10, I-TYPE), (11, 20, I-T..."
4,30,"[agnesi, пше]","[1, 3]","[8, 6084, 16805, 633, 454, 682, 2]","[1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 3, 3, -100]","[(0, 3, B-TYPE), (4, 10, I-TYPE), (11, 20, I-T..."
...,...,...,...,...,...,...,...
5445,27232,[яыц],[3],"[35, 19, 557, 2]","[1, 1, 1, 1]","[3, 3, 3, -100]","[(0, 3, B-TYPE), (4, 10, B-BRAND), (11, 20, I-..."
5446,27233,[яыца],[3],"[35, 19, 518, 2]","[1, 1, 1, 1]","[3, 3, 3, -100]","[(0, 3, B-TYPE), (4, 10, B-BRAND), (11, 20, I-..."
5447,27240,"[№1, газе]","[1, 3]","[545, 471, 3762, 13, 2]","[1, 1, 1, 1, 1]","[1, 1, 3, 3, -100]","[(0, 3, B-TYPE), (4, 10, B-BRAND), (11, 20, I-..."
5448,27247,"[№1, кофейник]","[1, 3]","[545, 471, 18465, 588, 2]","[1, 1, 1, 1, 1]","[1, 1, 3, 3, -100]","[(0, 3, B-TYPE), (4, 10, B-BRAND), (11, 20, I-..."


In [19]:
df_all_valid_docs.iloc[-1]['predicted_ner_tags']

[(0, 3, 'B-TYPE'), (4, 10, 'I-TYPE'), (11, 20, 'I-TYPE')]

## где ошибка в трейне

In [ ]:
df_valid_final[df_valid_final.annotation.astype(str) != df_valid_final.ann.astype(str)].to_csv('comparison_valid.csv', index=False)

In [ ]:
folds[0]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 21800
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5451
    })
})

In [ ]:
dd = pd.DataFrame()
dd.empty

True

## итоговая тренировка

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [34]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2',
                                          use_fast=True,
                                          add_prefix_space=True)

model = AutoModelForTokenClassification.from_pretrained('cointegrated/rubert-tiny2',
                                                        num_labels=len(lbls_in_dataset),
                                                        id2label=id2label,
                                                        label2id=label2id).to("cuda")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:

ds_train = ds.map(tokenize_and_align_labels,
                                          batched=True,
                                          fn_kwargs={'tokenizer': tokenizer})

training_args = TrainingArguments(
    eval_strategy="no",  # No evaluation during training
    torch_compile=True,
    per_device_train_batch_size=256,
    learning_rate=0.00037710330953711225,          # fixed value (was suggested by Optuna before)
    weight_decay=0.040411977783360725,
    num_train_epochs=6,          # fixed
    seed=42,
    data_seed=24,
    gradient_accumulation_steps=1,
    warmup_ratio=0.1,
    report_to=None,
    logging_dir="./logs",
    logging_steps=20,
    )

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    data_collator=data_collator,
    tokenizer=tokenizer,

)

# Start training
print("Starting training without evaluation...")
trainer.train()
trained_model = trainer.model




Map:   0%|          | 0/27552 [00:00<?, ? examples/s]

The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_36/1467123832.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training without evaluation...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
20,1.578600
40,0.612800
60,0.375000
80,0.290900
100,0.239900
120,0.194600
140,0.163800
160,0.164500
180,0.127400
200,0.105300


In [18]:
ds_train.select(range(5)).to_pandas()

,id,tokens,ner_tags,input_ids,attention_mask,labels
0,0,[aa],[0],"[351, 500, 2]","[1, 1, 1]","[0, 0, -100]"
1,1,[aala],[0],"[351, 700, 500, 2]","[1, 1, 1, 1]","[0, 0, 0, -100]"
2,2,[aarcca],[0],"[351, 1228, 679, 4701, 2]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, -100]"
3,3,[abon],[0],"[8, 4877, 1146, 2]","[1, 1, 1, 1]","[0, 0, 0, -100]"
4,4,[abso],[1],"[8, 4877, 12364, 2]","[1, 1, 1, 1]","[1, 1, 1, -100]"


In [36]:
import torch
import torch.nn.functional as F
import re

def predict_all_entities(text: str, model, tokenizer, id2label, device=None, debug=False):
    """
    Word-level inference with original character spans (start_idx, end_idx, ENTITY).
    Returns a list of tuples for each word (including 'O').
    """

    model.eval()
    if device is None:
        device = next(model.parameters()).device

    # --- find words and their char spans in original text ---
    words = []
    spans = []
    for match in re.finditer(r"\S+", text):
        words.append(match.group())
        spans.append(match.span())  # (start_idx, end_idx)

    # encode with word-level info
    enc = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)
    word_ids = enc.word_ids(batch_index=0)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[0]               # (seq_len, num_labels)
        probs = F.softmax(logits, dim=-1)        # (seq_len, num_labels)
    # print()
    # print(probs)
    results = []
    prev_word_idx = None
    # print(word_ids)
    for token_idx, word_idx in enumerate(word_ids):
        if debug:
            print(token_idx, word_idx, logits[token_idx])
        if word_idx is None:
            prev_word_idx = None
            continue

        # only take first subtoken per word
        if word_idx != prev_word_idx:
            label_id = int(torch.argmax(logits[token_idx]).cpu().numpy())
            label = id2label[label_id]

            start_idx, end_idx = spans[word_idx]
            results.append((start_idx, end_idx, label))

        prev_word_idx = word_idx

    return results


# -------------------------
# Example usage
# -------------------------
s = "сыр натура сливочный"

# Example id2label (replace with your mapping)
# id2label = {0: "O", 1: "B-TYPE", 2: "I-TYPE", ...}

res = predict_all_entities(s, trained_model, tokenizer, id2label)
print(res)


[(0, 3, 'B-TYPE'), (4, 10, 'O'), (11, 20, 'O')]


In [37]:
df_validation = pd.read_csv("/kaggle/input/submission/submission.csv", sep=";", usecols=['sample'])

In [38]:
predicted_ner_tags = []
for doc in tqdm(df_validation['sample'].tolist()):
    predicted_doc_ner = predict_all_entities(doc, trained_model, tokenizer, id2label)
    predicted_ner_tags.append(predicted_doc_ner)
df_validation['annotation'] = predicted_ner_tags

100%|██████████| 5000/5000 [00:13<00:00, 369.85it/s]


In [39]:
df_validation.to_csv('submission_rubert_tiny2_ft_augmented_new_6_epochs_260925.csv', index=False, sep=';')

In [22]:
predict_all_entities("garner ructi", trained_model, tokenizer, id2label, debug=True)

0 0 tensor([ 6.5906, 10.1643, -2.2721,  3.4366, -4.5278,  2.5592, -2.8466, -6.9580,
        -3.9529], device='cuda:0')
1 0 tensor([ 7.7309, 10.9714, -3.8946,  3.1987, -4.7314,  3.2854, -1.5130, -5.1659,
        -4.1112], device='cuda:0')
2 0 tensor([ 7.2921, 10.1729, -5.3683,  2.7412, -4.5518,  4.3491, -3.4186, -4.6163,
        -5.3117], device='cuda:0')
3 1 tensor([ 5.6538,  3.8720, -0.9463,  3.5795, -3.4112,  5.8417, -3.4994, -3.0971,
        -2.3954], device='cuda:0')
4 1 tensor([ 6.2706,  6.4004, -1.8007,  6.0896, -2.7667,  3.7546, -2.8255, -1.3013,
        -5.2245], device='cuda:0')
5 1 tensor([ 6.1525,  4.2112, -3.2930,  6.1238, -4.6684,  3.9089, -2.6030, -0.5535,
        -2.8735], device='cuda:0')
6 1 tensor([ 6.2605,  5.1263, -3.0759,  4.7641, -2.6424,  4.7713, -2.8157, -2.3314,
        -4.1695], device='cuda:0')
7 None tensor([ 1.7911, -0.1701,  0.5656, -0.5731,  1.6106, -0.8561,  2.0464,  2.4553,
         1.5692], device='cuda:0')


[(0, 6, 'B-BRAND'), (7, 12, 'I-BRAND')]

In [30]:
id2label

{0: 'O',
 1: 'B-BRAND',
 2: 'B-PERCENT',
 3: 'B-TYPE',
 4: 'B-VOLUME',
 5: 'I-BRAND',
 6: 'I-PERCENT',
 7: 'I-TYPE',
 8: 'I-VOLUME'}

In [31]:
tokenizer.tokenize(" garner ructi")

['▁', 'gar', 'ner', '▁', 'ru', 'c', 'ti']

In [82]:
# молоко 1,5 % -- не находит процент
# стейк говядина --  проверить усреднение
# сок 2 литра яблочный -- не находит volume 

In [37]:
predict_all_entities("schwartau со", trained_model, tokenizer, id2label, debug=True)

0 0 tensor([ 2.4230e+00,  1.3040e+01, -6.8861e-03,  9.9319e-01, -1.6785e+00,
        -1.4810e+00, -8.7815e-01, -2.5692e-01, -1.3998e+00], device='cuda:0')
1 0 tensor([ 2.2394, 12.3609,  0.4930,  1.3229, -1.7823, -0.0829, -1.3356, -0.2234,
        -0.8064], device='cuda:0')
2 0 tensor([ 2.6557e+00,  1.3508e+01,  1.0873e-02,  1.7207e+00, -2.1920e+00,
         4.2342e-01, -7.9274e-01, -9.1873e-02, -1.4111e+00], device='cuda:0')
3 0 tensor([ 2.6026, 13.6531, -0.9042,  1.6521, -2.1215,  0.8590, -0.8216, -1.2197,
        -0.8692], device='cuda:0')
4 0 tensor([ 3.3646, 14.1260,  0.9424,  1.7963, -0.5018,  1.6216, -1.6133, -2.3228,
        -1.5213], device='cuda:0')
5 1 tensor([ 6.6762, -1.6012, -2.8019, 11.6132, -3.1620,  1.1287, -1.9783,  0.8047,
        -2.5529], device='cuda:0')
6 None tensor([ 6.3755, -2.4741, -3.1129, 14.5177, -3.3633, -0.2019, -3.0747,  1.3072,
        -4.7479], device='cuda:0')


[(0, 9, 'B-BRAND'), (10, 12, 'B-TYPE')]

In [36]:
tokenizer.tokenize("крем jundo")

['▁крем', '▁j', 'und', 'o']

In [32]:
tokenizer2 = AutoTokenizer.from_pretrained('ai-forever/ruT5-large',
                                          use_fast=True,
                                          add_prefix_space=True)

model2 = AutoModel.from_pretrained('ai-forever/ruT5-large').to("cuda")

In [36]:
model2.encoder

T5Stack(
  (embed_tokens): Embedding(32128, 1024)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_features=1024, bias=False)
            (relative_attention_bias): Embedding(32, 16)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          (DenseReluDense): T5DenseActDense(
            (wi): Linear(in_features=1024, out_features=4096, bias=False)
            (wo): Linear(in_features=4096, out_features=1024, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
            (act): ReLU()
          )
          (layer_norm): T5LayerNorm()
 

In [39]:
trained_model

T5ForTokenClassification(
  (transformer): T5EncoderModel(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo):

In [41]:
trainer.save_model("/kaggle/working/rubert_tiny2_ft_optuna_260925")

In [ ]:
tr